In [172]:
import numpy as np
import pandas as pd
import string
from string import *
import pymongo
from collections import Counter
import matplotlib.pyplot as plt

In [286]:
mc = pymongo.MongoClient()
db = mc['allrecipes']
recipes_coll = db['recipes']
recipes = list(recipes_coll.find())

In [199]:
def find_ingredient_phrase(phrase):
    for recipe in recipes:
        raw_ings = recipe['ingredients_raw']
        for ing in raw_ings:
            if len(ing.split(phrase)) > 1:
                print('Ingredient line :\n', ing)
                
                print('\nEntire recipe :\n', recipe)

In [343]:
### CONSTANTS ###

units = ['pound', 'pounds', 'cup', 'cups', 'tablespoon', 'tablespoons', 'teaspoon', 'teaspoons',
         'clove', 'cloves', 'stalk', 'stalks', 'ounce', 'ounces', 'oz.', 'oz', 'cubes', 'pint', 'pints',
         'quart', 'quarts', 'dash', 'dashs', 'dashes', 'rib', 'ribs', 'bunch', 'bunches', 'pinch', 'head',
         'heads', 'slices', 'slice']

manual = ['2 to 3 pound', 'finely chopped from 1 can', 'onion soup, prepared from']

phrases = [' - ',', or ', ' for garnish', 'cut ', 'such as', ' like ', 'e.g.', 'with', ' or ', 'see note', 
           'to taste']

stopwords = ['and', 'into', 'very', 'hot', 'cold', 'warm', 'fresh', 'frozen', 'large', 'medium', 'small', 'halves', 'torn', 'bulk',
             'optional', 'fatfree', 'lowsodium', 'low', 'sodium', 'reducedsodium', 'reducedfat', 'ripe', 'lean',
             'extra', 'pure', 'goya', 'whole', 'ground', 'domestic', 'extravirgin']

suffixes = ['ed','less','ly']

flag_words = ['can or bottle', 'can', 'cans', 'package', 'packages', 'jar', 'jars', 'container', 'containers', 'bag', 'bags',
              'bottle', 'bottles', 'envelope', 'envelopes', 'carton','cartons', 'packet', 'packets']
flag_words.sort(key=len)
flag_words.reverse()

identicals = {'bell pepper':['green bell pepper', 'red bell pepper', 'yellow bell pepper', 'orange bell pepper'],
              'chicken': ['whole chicken', 'chicken breast', 'chicken thigh'],
              'onion': ['yellow onion', 'white onion', 'sweet onion', 'red onion'],
              'kidney beans': ['dark red kidney beans', 'light red kidney beans'],
              'bay leaf': ['bay leave'],
              'half-and-half': ['halfandhalf', 'halfandhalf cream','halfhalf'],
              'corn' : ['kernel corn', 'corn kernel'],
              'ginger' : ['ginger root'],
              'salt' : ['kosher salt', 'sea salt'],
              'beef' : ['beef stew meat'],
              'heavy cream' : ['heavy whipping cream'],
              'vegetable oil' : ['canola oil'],
              'green chile' : ['green chile pepper'],
              'rice' : ['white rice'],
              'egg' : ['eggs'],
              'garlic' : ['garlic clove'],
              'basil' : ['basil leave']}

conversion_dict = {}
conversion_dict['ounce'] = {'other':1}
conversion_dict['cup'] = {'other':8}
conversion_dict['pint'] = {'other':16}
conversion_dict['quart'] = {'other':32}
conversion_dict['gallon'] = {'other':128}
conversion_dict['fluid ounce'] = {'other':1}
conversion_dict['milliliter'] = {'other':0.034}
conversion_dict['pound'] = {'other': 16}
conversion_dict['tablespoon'] = {'other': 1/2}
conversion_dict['teaspoon'] = {'other': 1/6}
conversion_dict['pinch'] = {'other': 0.1}
conversion_dict['dash'] = {'other': 0.1}

conversion_dict['bunch'] =  {'green onion': 3,
                             'cilantro': 2.8,
                             'parsley': 2,
                             'other': 3}
conversion_dict['bunche'] = conversion_dict['bunch']
conversion_dict['clove'] = {'garlic': 0.5, 'other': 0.5}
conversion_dict['cube'] = {'chicken bouillon': 0.4,
                           'beef bouillon': 0.4,
                           'vegetable bouillon': 0.4,
                           'other': 0.4}
conversion_dict['packet'] = {'other':1}
conversion_dict['head'] = {'other': 20,
                             'escarole': 10,
                             'garlic clove': 1.5,
                             'garlic': 1.5,
                             'cabbage': 30,
                             'cauliflower': 30,
                             'broccoli': 20}
conversion_dict['rib'] = {'celery': 2, 'other': 2}
conversion_dict['stalk'] = {'celery': 2, 'other': 2}
conversion_dict['slice'] = {'bacon': 1, 'bread': 1, 'other': 1}
conversion_dict['each'] = {'onion': 8,
                             'bell pepper': 6,
                             'potato': 6,
                             'carrot': 4,
                             'jalapeno pepper': 0.7,
                             'chicken': 10,
                             'bay leaf': 1,
                             'tomato': 6,
                             'green onion': 0.5,
                             'zucchini': 5,
                             'bay leave': 1,
                             'lime': 1.5,
                             'head cabbage': 30,
                             'habanero pepper': 0.1,
                             'sweet potato': 6,
                             'eggs': 1,
                             'green chile pepper': 1,
                             'other': 1}

ing_categories = {'vegetable': ['onion','bell pepper','potato', 'carrot','jalapeno pepper','zucchini',
                                'tomato','celery','garlic','tomato sauce','parsley','cilantro','green onion',
                                'tomato paste','kidney beans','black beans','mushroom','corn','green chile',
                                'pinto beans','spinach','sweet potato','shallot','okra','chili beans',
                                'great northern beans','green beans','cannellini beans','butternut squash',
                                'apple','habanero pepper','peas','garbanzo beans','leek','cabbage'],
                  'protein'  : ['chicken','beef','shrimp','bacon','turkey','ham','andouille sausage','egg','sausage',
                                'italian sausage'],
                  'base'     : ['water', 'chicken broth', 'milk','coconut milk','chicken stock', 'heavy cream',
                                'vegetable broth','beef broth','halfandhalf','cream of chicken soup','tomato soup'],
                  'seasoning': ['salt','black pepper','cumin','chili powder','cayenne pepper','curry powder',
                                'oregano','white sugar','thyme','paprika','basil','garlic powder','bay leaf',
                                'red pepper flake','brown sugar','worcestershire sauce','ginger','lemon juice',
                                'turmeric','soy sauce','cinnamon','coriander','pepper sauce','lime juice',
                                'white vinegar','clove','onion powder','garam masala','lime','white pepper',
                                'chicken bouillon','fish sauce','cajun seasoning','honey','rosemary','ketchup',
                                'nutmeg','red curry paste','taco seasoning mix','salsa','italian seasoning',
                                'sage','red pepper','cumin seed','dill','allspice','garlic salt'],
                  'fat'      : ['butter','olive oil','vegetable oil','margarine','vegetable shortening','peanut oil'],
                  'other'    : ['allpurpose flour','cheddar cheese','sour cream','parmesan cheese','cream cheese',
                                'cornstarch','rice','monterey jack cheese','red wine','white wine','mayonnaise',
                                'beer','sauce','raisin']}


### MAIN FUNCTIONS ###

def parse_ingredients(ingredients, units=units, flag_words=flag_words):
    '''
    Parses a list of ingredients into a list of dictionaries with the following format: 
        {'quantity': (float),
         'units': (str),
         'ingredient': (str)}
    Also takes argument 'units', a list of accepted units (e.g., ['cups', 'tablespoon']).
    If an ingredident does not specify a unit in this list, the label 'each' will be applied.
    '''
    ing_dict = {}
    for item in ingredients:
        item_dict = {}
        # Check item for flag words/phrases(require special parsing treatment)
        manual_flag = False
        for man_phrase in manual:
            if len(item.split(man_phrase)) > 1:
                manual_flag = True
        sp_flag = False
        for word in item.split():
            if word in flag_words:
                f_word = word
                sp_flag = True
        if item.split()[1][0] == '(':
            f_word = '('
            sp_flag = True  
        # Parse quantities and units        
        if manual_flag:
            quantity, unit, remainder = _parse_manual(item)
            item_dict['quantity'] = quantity
            item_dict['units'] = unit if unit[-1] != 's' else unit[:-1]
        elif sp_flag:
            try:
                quantity, unit, remainder = _parse_special(item, flag_words)                
                item_dict['quantity'] = quantity
                item_dict['units'] = unit if unit[-1] != 's' else unit[:-1]
            except:
                # Exception for special units of unspecified size
                item_dict['quantity'] = float(item.split()[0])
                item_dict['units'] = f_word if f_word[-1] != 's' else f_word[:-1]
                remainder = ' '.join(item.split()[2:])
        else:
            quantity, remainder = _determine_quantity(item) 
            item_dict['quantity'] = quantity
            if remainder.split()[0] in units:
                unit = remainder.split()[0]
                item_dict['units'] = unit if unit[-1] != 's' else unit[:-1]
                remainder = ' '.join(remainder.split()[1:])
            else:
                item_dict['units'] = 'each'
        # Remove preparation instructions from remaining text to isolate ingredient
        parsed = _remove_descriptors(remainder)
        if not parsed:
            continue
        ing_name = _merge_identicals(parsed, identicals)
        item_dict['normalized_qty'] = _normalize_ingredient_quantity(ing_name, item_dict, conversion_dict)
        item_dict['category'] = _get_ingredient_category(ing_name, ing_categories)
        # Add item dictionary to ingredient dictionary
        # If duplicate, combine normalized quantities and flag other entries
        if ing_name in ing_dict.keys():
            qty = ing_dict[ing_name]['normalized_qty']
            item_dict['normalized_qty'] += qty
            item_dict['units'] = 'combined'
            item_dict['quantity'] = -1
            ing_dict[ing_name] = item_dict
        else:
            ing_dict[ing_name] = item_dict
        
    return ing_dict


### HELPER FUNCTIONS ###

def _determine_quantity(item):
    quantity = 0
    for i, elem in enumerate(item.split()):
        if elem[0] in string.digits + '.':
            try:
                quantity += float(elem)
            except:
                numer, denom = elem.split('/')
                quantity += float(numer) / float(denom)
        else:
            idx = i
            break
    remainder = ' '.join(item.split()[idx:])
    return quantity, remainder

def _parse_special(item, flag_words):
    # Determine special word
    sp_word = ')'
    for word in flag_words:
        if len(item.split(word)) > 1:
            sp_word = ' ' + word + ' '
            break
    
    # Parse item 
    count_and_size = item.split(sp_word)[0]
    remainder = item.split(sp_word)[1]
    count, rest = _determine_quantity(count_and_size)
    if sp_word == ')':
        size, unit = _determine_quantity(rest[1:])
    else:
        size, unit = _determine_quantity(rest[1:-1])
    quantity = count * size
    return quantity, unit, remainder

def _parse_manual(item):
    if len(item.split('2 to 3 pound')) > 1:
        quantity = float(item.split()[0]) * 2.5
        unit = 'pound'
        remainder = item.split('2 to 3 pound')[1]
        return quantity, unit, remainder
    if len(item.split('finely chopped from 1 can')) > 1:
        quantity = 1.0
        unit = 'ounce'
        remainder = 'chipotle chile'
        return quantity, unit, remainder
    if len(item.split('onion soup, prepared from')) > 1:
        quantity = 1.5
        unit = 'cup'
        remainder = 'onion soup'
        return quantity, unit, remainder

def _remove_descriptors(item,
                        phrases=phrases,
                        stopwords=stopwords,
                        suffixes=suffixes):
    # Remove common/unnecessary ending phrases
    for phrase in phrases:
        if len(item.split(phrase)) > 1:
            item = item.split(phrase)[0]
    # Remove punctuation and stopwords
    words = []
    for elem in item.split():
        word = ''.join([letter for letter in elem.lower() if letter in string.ascii_lowercase])
        if word not in stopwords:
            words.append(word)
    # Remove adjectives and adverbs    
    for suffix in suffixes:
        for word in words.copy():
            try:
                if (word[-len(suffix):] == suffix) and word not in ['red', 'shortening']:
                    words.remove(word)
            except:
                continue    
    # Remove trailing spaces
    result = ' '.join([word for word in words if word])
    # Singularize (when not beans)...also, this code is asinine
    if result[-3:] == 'oes':
        result = result[:-2]
    if len(result) < 5:
        return result
    if result[-5:] == 'beans':
        return result
    else:
        return result if result[-1] != 's' else result[:-1]
    
def _normalize_ingredient_quantity(ing_name, item_dict, conversion_dict):
    qty = item_dict['quantity']
    units = item_dict['units']
    if units in conversion_dict.keys():
        conv_factor_dict = conversion_dict[units]
        if ing_name in conv_factor_dict.keys():
            conv_factor = conv_factor_dict[ing_name]
        else:
            conv_factor = conv_factor_dict['other']
        return qty * conv_factor
    else:
        return qty
    
def _merge_identicals(ingredient, identicals):
    for key, val in identicals.items():
        if ingredient in val:
            return key
    return ingredient

def _get_ingredient_category(ingredient, category_dict):
    for key, val in category_dict.items():
        if ingredient in val:
            return key
    return 'other'

In [337]:
recipe = recipes[10]
raw_ing = recipe['ingredients_raw']
raw_ing

['2 1/2 pounds lean ground beef',
 '1 medium onion, chopped',
 '1 green bell pepper, seeded and chopped (optional)',
 '3 cloves garlic, pressed',
 '1/4 cup Worcestershire sauce',
 '5 tablespoons chili powder',
 '2 teaspoons ground cumin',
 '2 teaspoons dried oregano',
 '1 (15 ounce) can kidney beans, rinsed and drained',
 '1 (15 ounce) can cannellini beans, rinsed and drained',
 '2 (12 ounce) bottles chile sauce',
 '1 (14 ounce) can beef broth',
 '2 cups shredded Cheddar cheese',
 '1/4 cup chopped jalapeno pepper (optional)']

In [338]:
pd.DataFrame(parse_ingredients(raw_ing))

,beef,onion,bell pepper,garlic,worcestershire sauce,chili powder,cumin,oregano,kidney beans,cannellini beans,chile sauce,beef broth,cheddar cheese,jalapeno pepper
normalized_qty,40,8,6,1.5,2,2.5,0.333333,0.333333,15,15,24,14,16,2
quantity,2.5,1,1,3,0.25,5,2,2,15,15,24,14,2,0.25
units,pound,each,each,clove,cup,tablespoon,teaspoon,teaspoon,ounce,ounce,ounce,ounce,cup,cup


In [327]:
ingredients = Counter()
for recipe in recipes:
    ing_list = recipe['ingredients_raw']
    parsed = parse_ingredients(ing_list)
    for ing in parsed.keys():
        ingredients[ing] += 1
print('Number of unique ingredients :', len(ingredients))

Number of unique ingredients : 1669


In [328]:
ingredients.most_common()

[('onion', 925),
 ('garlic', 614),
 ('salt', 441),
 ('tomato', 405),
 ('water', 401),
 ('black pepper', 326),
 ('chicken broth', 304),
 ('celery', 289),
 ('cumin', 276),
 ('butter', 262),
 ('bell pepper', 260),
 ('olive oil', 259),
 ('carrot', 254),
 ('chili powder', 246),
 ('chicken', 230),
 ('vegetable oil', 214),
 ('allpurpose flour', 211),
 ('cayenne pepper', 196),
 ('curry powder', 159),
 ('potato', 158),
 ('oregano', 151),
 ('beef', 144),
 ('parsley', 143),
 ('tomato sauce', 130),
 ('cilantro', 126),
 ('milk', 118),
 ('white sugar', 115),
 ('thyme', 115),
 ('green onion', 109),
 ('paprika', 105),
 ('tomato paste', 100),
 ('basil', 100),
 ('garlic powder', 97),
 ('bay leaf', 96),
 ('kidney beans', 94),
 ('red pepper flake', 87),
 ('shrimp', 85),
 ('coconut milk', 81),
 ('jalapeno pepper', 75),
 ('black beans', 72),
 ('brown sugar', 72),
 ('chicken stock', 71),
 ('cheddar cheese', 70),
 ('bacon', 68),
 ('worcestershire sauce', 65),
 ('ginger', 65),
 ('bay leave', 64),
 ('lemon juic

In [329]:
common_ingredients = []
for item, count in ingredients.items():
    if count > 1:
        common_ingredients.append(item)
print('Number of common ingredients :', len(common_ingredients))

Number of common ingredients : 704


In [330]:
ing_units = {}
unit_ings = {}
num_units = {}
all_units = set()
for recipe in recipes:
    ing_list = recipe['ingredients_raw']
    parsed = parse_ingredients(ing_list)
    for ing, val in parsed.items():
        all_units.add(val['units'])
        # For each ingredient, count the occurences of different units
        units_counter = ing_units.get(ing, Counter())
        units_counter[val['units']] += 1
        ing_units[ing] = units_counter
        # For each unit, count the occurences of different ingredients
        ings_counter = unit_ings.get(val['units'], Counter())
        ings_counter[ing] += 1
        unit_ings[val['units']] = ings_counter
for item, counter in ing_units.items():
        num_units[item] = len(counter)

In [331]:
unit_ings['slice'].most_common()

[('bacon', 40),
 ('bread', 6),
 ('ginger root', 3),
 ('provolone cheese', 3),
 ('ginger', 2),
 ('swiss cheese', 2),
 ('french bread', 2),
 ('american cheese', 2),
 ('onion', 2),
 ('gruyere', 1),
 ('dayold bread', 1),
 ('smithfield thick cut bacon', 1),
 ('toast', 1),
 ('bacon strip', 1),
 ('galangal', 1),
 ('crisp bacon', 1),
 ('white bread', 1),
 ('italian bread', 1),
 ('tomato', 1),
 ('thick bacon', 1),
 ('mozzarella cheese', 1),
 ('hearty farmhousestyle bread', 1),
 ('of french bread', 1),
 ('pancetta bacon', 1),
 ('habanero pepper', 1)]

In [324]:
all_units

{'(',
 'bunch',
 'bunche',
 'clove',
 'combined',
 'cube',
 'cup',
 'dash',
 'dashe',
 'each',
 'envelope',
 'fluid ounce',
 'head',
 'inch',
 'inch square',
 'inch thick',
 'milliliter',
 'oun',
 'ounce',
 'packet',
 'pinch',
 'pint',
 'pound',
 'quart',
 'rib',
 'slice',
 'stalk',
 'tablespoon',
 'teaspoon',
 'to 3 1/2 pound'}

In [271]:
conversion_dict = {}
conversion_dict['ounce'] = {'other':1}
conversion_dict['cup'] = {'other':8}
conversion_dict['pint'] = {'other':16}
conversion_dict['quart'] = {'other':32}
conversion_dict['gallon'] = {'other':128}
conversion_dict['fluid ounce'] = {'other':1}
conversion_dict['milliliter'] = {'other':0.034}
conversion_dict['pound'] = {'other': 16}
conversion_dict['tablespoon'] = {'other': 1/2}
conversion_dict['teaspoon'] = {'other': 1/6}
conversion_dict['pinch'] = {'other': 0.1}
conversion_dict['dash'] = {'other': 0.1}

conversion_dict['bunch'] =  {'green onion': 3,
                             'cilantro': 2.8,
                             'parsley': 2,
                             'other': 3}
conversion_dict['bunche'] = conversion_dict['bunch']
conversion_dict['clove'] = {'garlic': 0.5, 'other': 0.5}
conversion_dict['cube'] = {'chicken bouillon': 0.4,
                           'beef bouillon': 0.4,
                           'vegetable bouillon': 0.4,
                           'other': 0.4}
conversion_dict['packet'] = {'other':1}
conversion_dict['head'] = {'other': 20,
                             'escarole': 10,
                             'garlic clove': 1.5,
                             'cabbage': 30,
                             'cauliflower': 30,
                             'broccoli': 20}
conversion_dict['rib'] = {'celery': 2, 'other': 2}
conversion_dict['stalk'] = {'celery': 2, 'other': 2}
conversion_dict['slice'] = {'bacon': 1, 'bread': 1, 'other': 1}
conversion_dict['each'] = {'onion': 8,
                             'green bell pepper': 6,
                             'potato': 6,
                             'carrot': 4,
                             'red bell pepper': 6,
                             'jalapeno pepper': 0.7,
                             'chicken breast': 10,
                             'bay leaf': 1,
                             'yellow onion': 8,
                             'tomato': 6,
                             'green onion': 0.5,
                             'zucchini': 5,
                             'bay leave': 1,
                             'red onion': 8,
                             'yellow bell pepper': 6,
                             'slices bacon': 1,
                             'lime': 1.5,
                             'head cabbage': 30,
                             'sweet onion': 8,
                             'habanero pepper': 0.1,
                             'sweet potato': 6,
                             'eggs': 1,
                             'green chile pepper': 1,
                             'white onion': 8,
                             'other': 1}

In [295]:
find_ingredient_phrase('to 3 1/2 pound')

Ingredient line :
 2 (3 to 3 1/2 pound) domestic whole ducklings, dressed

Entire recipe :
 {'_id': ObjectId('5ca65149c6d0b01738f794e8'), 'id': 133040, 'name': "Teddy's Duck Gumbo", 'href': 'https://www.allrecipes.com/recipe/133040/teddys-duck-gumbo/', 'category': {'lvl_1': 'Soups, Stews and Chili', 'lvl_2': 'Stews', 'lvl_3': 'Gumbo'}, 'rating_info': None, 'submitter_info': {'id': 2239023, 'name': 'teddynpooh', 'followers': 0, 'href': 'https://www.allrecipes.com/cook/2239023/'}, 'ingredients': [{'quantity': 6.0, 'units': 'to 3 1/2 pound', 'ingredient': 'domestic duckling', 'normalized_qty': 6.0}, {'quantity': 3.0, 'units': 'cup', 'ingredient': 'water', 'normalized_qty': 24.0}, {'quantity': 16.0, 'units': 'ounce', 'ingredient': 'tomato', 'normalized_qty': 16.0}, {'quantity': 6.0, 'units': 'clove', 'ingredient': 'garlic', 'normalized_qty': 3.0}, {'quantity': 1.5, 'units': 'cup', 'ingredient': 'onion', 'normalized_qty': 12.0}, {'quantity': 1.5, 'units': 'cup', 'ingredient': 'celery', 'nor

In [342]:
ing_units['basil leave']

Counter({'teaspoon': 4, 'tablespoon': 4, 'cup': 8, 'each': 2})

In [209]:
'10.75 ounce'.split()

['10.75', 'ounce']

In [219]:
_determine_quantity('.75 ounce')

(0.75, 'ounce')

In [227]:
s = '1 (28 ounce) can whole tomatoes, chopped'
_parse_special(s, flag_words=flag_words)

(28.0, 'ounce', 'whole tomatoes, chopped')

In [272]:
def _normalize_ingredient_quantity(ingredient_dict, conversion_dict):
    ing = ingredient_dict['ingredient']
    qty = ingredient_dict['quantity']
    units = ingredient_dict['units']
    if units in conversion_dict.keys():
        conv_factor_dict = conversion_dict[units]
        if ing in conv_factor_dict.keys():
            conv_factor = conv_factor_dict[ing]
        else:
            conv_factor = conv_factor_dict['other']
        return qty * conv_factor
    else:
        return qty

In [274]:
ex = {'quantity': 0.5, 'units': 'teaspoon', 'ingredient': 'ground black pepper'}
_normalize_ingredient_quantity(ex, conversion_dict)

0.08333333333333333

In [294]:
recipes[1000]

{'_id': ObjectId('5ca64f88c6d0b016779db25b'),
 'id': 256617,
 'name': 'Serrano Hot Sauce',
 'href': 'https://www.allrecipes.com/recipe/256617/serrano-hot-sauce/',
 'category': {'lvl_1': 'Side Dish',
  'lvl_2': 'Sauces and Condiments',
  'lvl_3': None},
 'rating_info': None,
 'submitter_info': {'id': 233956,
  'name': 'Jon Rachiele',
  'followers': 7,
  'href': 'https://www.allrecipes.com/cook/233956/'},
 'ingredients': [{'quantity': 14.0,
   'units': 'each',
   'ingredient': 'serrano chile pepper',
   'normalized_qty': 14.0},
  {'quantity': 6.0,
   'units': 'clove',
   'ingredient': 'garlic',
   'normalized_qty': 3.0},
  {'quantity': 1.5,
   'units': 'tablespoon',
   'ingredient': 'kosher salt',
   'normalized_qty': 0.75},
  {'quantity': 1.5,
   'units': 'cup',
   'ingredient': 'white vinegar',
   'normalized_qty': 12.0}],
 'ingredients_raw': ['14 serrano chile peppers, stemmed',
  '6 cloves garlic, smashed',
  '1 1/2 tablespoons kosher salt',
  '1 1/2 cups distilled white vinegar'],
 